# Reading groundwater observations

This notebook introduces how to use the `hydropandas` package to read, process and visualise groundwater data from Dino and Bro databases.

## <a id=top></a>Notebook contents

1. [GroundwaterObs](#GroundwaterObs)
2. [ObsCollections](#ObsCollections)
3. [Read ObsCollections](#Read-ObsCollections)
4. [Write ObsCollections](#Write-ObsCollections)

In [ ]:
from IPython.display import HTML

import hydropandas as hpd

In [ ]:
hpd.util.get_color_logger("INFO")

## GroundwaterObs

The hydropandas package has several functions to read groundwater observations at a measurement well. These include reading data from:
- dino (from csv-files).
- bro (using the bro-api)
- fews (xml dumps from the fews database)
- wiski (dumps from the wiski database)

In [ ]:
# reading a dino csv file
path = "data/Grondwaterstanden_Put/B33F0080001_1.csv"
gw_dino = hpd.GroundwaterObs.from_dino(path=path)
gw_dino

In [ ]:
# reading the same filter from using the bro api. Specify a groundwater monitoring id (GMW00...) and a filter number (1)
gw_bro = hpd.GroundwaterObs.from_bro("GMW000000041261", 1)

Now we have an `GroundwaterObs` object named `gw_bro` and `gw_dino`. Both objects are from the same measurement well in different databases. A `GroundwaterObs` object inherits from a pandas `DataFrame` and has the same attributes and methods.

In [ ]:
gw_bro.describe()

In [ ]:
gw_bro

In [ ]:
ax = gw_dino["stand_m_tov_nap"].plot(
    label="dinoloket", figsize=(14, 5), legend=True, marker=".", lw=0.2
)
gw_bro["values"].plot(ax=ax, label="bro", legend=True, ylabel=gw_bro.unit)
gw_dino["ground_level"].plot(
    ax=ax,
    label="ground level",
    legend=True,
    grid=True,
    color="green",
    ylabel=gw_dino.unit,
)

ax.set_title(f"same tube from Dinoloket {gw_dino.name} and BRO id {gw_bro.name}")

#### GroundwaterObs Attributes

Besides the standard `DataFrame` attributes a `GroundwaterObs` has the following additional attributes:
- x, y: x- and y-coordinates of the observation point
- name: str with the name
- filename: str with the filename (only available when the data was loaded from a file)
- monitoring_well: the name of the monitoring_well. One monitoring well can have multiple tubes.
- tube_nr: the number of the tube. The combination of monitoring_well and tube_nr should be unique
- screen_top: the top of the tube screen (bovenkant filter in Dutch)
- screen_bottom: the bottom of the tube screen (onderkant filter in Dutch)
- ground_level: surface level (maaiveld in Dutch)
- tube_top: the top of the tube
- metadata_available: boolean indicating whether metadata is available for this observation point
- meta: dictionary with additional metadata

When dowloading from Dinoloket all levels are in meters NAP.

In [ ]:
print(gw_bro)

#### GroundwaterObs methods

Besides the standard `DataFrame` methods a `GroundwaterObs` has additional methods. This methods are accessible through submodules:
- `geo.get_lat_lon()`, to obtain latitude and longitude
- `gwobs.get_modellayer()`, to obtain the modellayer of a modflow model using the filter depth
- `stats.get_seasonal_stat()`, to obtain seasonal statistics
- `stats.obs_per_year()`, to obtain the number of observations per year
- `stats.consecutive_obs_years()`, to obtain the number of consecutive years with more than a minimum number of observations
- `plots.interactive_plot()`, to obtain a bokeh plot





Get latitude and longitude with `gw.geo.get_lat_lon()`:

In [ ]:
print(f"latitude and longitude -> {gw_bro.geo.get_lat_lon()}")

In [ ]:
gw_bro.stats.get_seasonal_stat(stat="mean")

In [ ]:
p = gw_bro.plots.interactive_plot("figure")
HTML(filename="figure/{}.html".format(gw_bro.name))

## ObsCollections

`ObsCollections` are a combination of multiple observation objects. The easiest way to construct an `ObsCollections` is from a list of observation objects.

In [ ]:
path1 = "data/Grondwaterstanden_Put/B33F0080001_1.csv"
path2 = "data/Grondwaterstanden_Put/B33F0133001_1.csv"
gw1 = hpd.GroundwaterObs.from_dino(path=path1)
gw2 = hpd.GroundwaterObs.from_dino(path=path2)

# create ObsCollection
oc = hpd.ObsCollection([gw1, gw2], name="Dino groundwater")
oc

Now we have an `ObsCollection` object named `oc`. The `ObsCollection` contains all the data from the two `GroundwaterObs` objects. It also stores a reference to the `GroundwaterObs` objects in the 'obs' column. An `ObsCollection` object also inherits from a pandas `DataFrame` and has the same attributes and methods.

In [ ]:
# get columns
oc.columns

In [ ]:
# get individual GroundwaterObs object from an ObsCollection
o = oc.loc["B33F0133-001", "obs"]
o

In [ ]:
# get statistics
oc.describe()

#### ObsCollection methods
Besides the methods of a pandas `DataFrame` an `ObsCollection` has additional methods stored in submodules.

`geo`:
- `get_bounding_box` -> get a tuple with (xmin, ymin, xmax, ymax)
- `get_extent` -> get a tule with (xmin, xmax, ymin, ymax)
- `get_lat_lon` -> to get the lattitudes and longitudes from the x and y coordinates
- `within_polygon` -> to select only the observations that lie within a polygon

`gwobs`:
- `set_tube_nr` -> to set the tube numbers based on the tube screen depth when there are multiple tubes at one monitoring well
- `set_tube_nr_monitoring_well` -> find out which observations are at the same location with a different screen depth. Set monitoring_well and tube_nr attributes accordingly.

`plots`:
- `interactive_figures` -> create bokeh figures for each observation point.
- `interactive_map` -> create a folium map with observation points and bokeh figures for each observation point.
- `section_plot` -> create a plot of multiple observations and a plot of the well layout.

`stats`:
- `get_first_last_obs_date()` -> get the first and the last date of the observations for each observation point
- `get_no_of_observations()` -> get the number of observations
- `get_seasonal_stat()` -> get seasonal stats of the observations

E.g. get the bounding box with `gw.geo.get_bounding_box()`:

In [ ]:
print(f"bounding box -> {oc.geo.get_bounding_box()}")

In [ ]:
oc.geo.set_lat_lon()
oc.plots.interactive_map(plot_dir="figure")

We can get an overview of the well layout and observations via `plots.section_plot`:

In [ ]:
oc.plots.section_plot()

#### ObsCollection Attributes

An `ObsCollection` also has additional attributes:
- name, a str with the name of the collection
- meta, a dictionary with additional metadata

In [ ]:
print(f"name is -> {oc.name}")
print(f"meta is -> {oc.meta}")

## Read ObsCollections

Instead of creating the ObsCollection from a list of observation objects. It is also possible to read the data from a source into an ObsCollection at once. The following sources can be read as an ObsCollection:

- bro (using the api)
- dino (from files)
- fews (dumps from the fews database)
- wiski (dumps from the wiski database)
- menyanthes (a .men file)
- modflow (from the heads of a modflow model)
- imod (from the heads of an imod model)

This notebook won't go into detail on all the sources that can be read. Only the two options for reading data from Dino and BRO are shown below.

In [ ]:
# read using a .zip file with data
dinozip = "data/dino.zip"
dino_gw = hpd.read_dino(
    dirname=dinozip, subdir="Grondwaterstanden_Put", suffix="1.csv", keep_all_obs=False
)
dino_gw

In [ ]:
ax = dino_gw.to_gdf().plot()
import contextily as ctx
ctx.add_basemap(ax=ax, crs=28992)

In [ ]:
# read from bro using an extent (Schoonhoven zuid-west)
oc = hpd.read_bro(extent=(117850, 118180, 439550, 439900), keep_all_obs=False)
oc

In [ ]:
# plot wells, use x-coordinate in section plot
oc.plots.section_plot(section_colname_x="x", section_label_x="x coordinate [m]")

In [ ]:
ax = oc.to_gdf().plot()
import contextily as ctx
ctx.add_basemap(ax=ax, crs=28992)

## Write ObsCollections

Sometimes reading ObsCollections can be time consuming, especially when you need to download a lot of data. It can be worth to save the ObsCollection to a file and read it later instead of going through the full read process again. There are two basic ways to do this:
1. Write the ObsCollection to a pickle
2. Write the ObsCollection to an excel file

|                | pickle | excel                                       |
|----------------|--------|---------------------------------------------|
| extension      | .pklz  | .xlsx                                       |
| human readable | No     | Yes                                         |
| data lost      | None   | Some metadata, see `hpd.to_excel` docstring |

##### Pickle
Pickling is used to store Python objects into a binary file that is not human readable. Writing and reading a pickle is fast and returns an exact copy of the ObsCollection. Exchanging pickles between machines can be troublesome because of machine settings and differences between package versions.

In [ ]:
oc.to_pickle("test.pklz")

In [ ]:
oc_pickled = hpd.read_pickle("test.pklz")

##### Excel
An ObsCollection can be written to Excel file. An Excel file with multiple sheets is created. One sheet with the metdata and another sheet for each observation in the ObsCollection. Writing to an excel file is considerably slower than writing a pickle but it does give you a human readable file format that can be easily exchanged between machines.

In [ ]:
oc.to_excel("test.xlsx")

In [ ]:
oc_excelled = hpd.read_excel("test.xlsx")